### Collect keypoints 1

In [1]:
!pip install tensorflow opencv-python mediapipe matplotlib numpy

In [2]:
import constants as cs
import globali as gg
import config_loader as cl
import detect_keypoints as dk

In [3]:
import cv2
import numpy as np
import os
import shutil

### Initialization

In [4]:
# initialization
gg.actions, gg.counters = cl.read_yaml_fields()

### Collect Data

In [5]:
# function that erase content videos' folder
def clean_folder():
    for file in os.listdir(cs.VIDEOS_FOLDER):
        file_path = os.path.join(cs.VIDEOS_FOLDER, file)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
            
# function to collect videos
def collect_data(action, no_sequences, no_frames, time_record, signer, camId):
    gg.actions, gg.counters = cl.read_yaml_fields()
    
    if action not in gg.actions:
        gg.counters[action] = 0
    
    action_folder = os.path.join(cs.KEYPOINTS_FOLDER, action)
    os.makedirs(action_folder, exist_ok=True)
    action_folder2 = os.path.join(cs.VIDEOS_FOLDER, action)
    os.makedirs(action_folder2, exist_ok=True)
    gg.actions.add(action)
    
    # clean folder videos
    #clean_folder()
    
    cap = cv2.VideoCapture(camId)
    
    # Set mediapipe model 
    with dk.mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
        for sequence in range(no_sequences):
            
            video_folder = os.path.join(action_folder, str(gg.counters[action] + sequence))
            os.makedirs(video_folder, exist_ok=True)
            video_folder2 = os.path.join(action_folder2, str(gg.counters[action] + sequence))
            os.makedirs(video_folder2, exist_ok=True)
            
            for frame_num in range(no_frames):
                
                ret, frame = cap.read()                                                     # Read feed
                image, results = dk.mediapipe_detection(frame, holistic)                   # Make detections
                dk.draw_styled_landmarks(image, results)                                   # Draw landmarks     
                
                num = gg.counters[action] + sequence
                frame_path = os.path.join(cs.VIDEOS_FOLDER, action, str(num), str(frame_num) + ".jpg")
                cv2.imwrite(frame_path, frame)
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(time_record)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                keypoints = dk.extract_keypoints(results)
                
                npy_path = os.path.join(video_folder, str(frame_num))
                np.save(npy_path, keypoints)
                
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                
        # update actions and counters
        gg.counters[action] += no_sequences
        cl.update_yaml_fields(gg.actions, gg.counters)
        
        cap.release()
        cv2.destroyAllWindows()
    cap.release()
    cv2.destroyAllWindows()    
    

In [19]:
# function to collect keypoints from webcam video
collect_data("candy", 30, cs.NUM_FRAME, 1500, "paolo", 0)                       